In [2]:
import cv2
import numpy as np
import os
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
#path=os.path.join(os.path.dirname(os.getcwd()),"raw_data","pushups.mp4")

In [3]:
def camera_test():
    cap = cv2.VideoCapture("../media/full_pushup.mp4")
    while(cap.isOpened()):
        ret, frame = cap.read()

        if not ret:
            break

        cv2.imshow('Video', frame)
        if cv2.waitKey(75) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    return "Successfully opened the camera"

In [5]:
camera_test()

'Successfully opened the camera'

# Top Left Text Box

In [12]:
def text_box_test():
    cap = cv2.VideoCapture("../media/full_pushup.mp4")
    with mp.solutions.pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while(cap.isOpened()):
            ret, frame = cap.read()
            
            if not ret:
                break

            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make detection
            results = pose.process(image)

            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
            # Setup status box
            cv2.rectangle(frame, (0,0), (225,225), (0,0,0), -1)

            # Status data
            cv2.putText(frame, 'Quality', (15,12),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (225,225,225), 1, cv2.LINE_AA)
            cv2.putText(frame, 'GOOD',
                        (10,60),
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
            cv2.imshow('Video', frame)
            if cv2.waitKey(75) & 0xFF == ord('q'):
                break
    cap.release()
    cv2.destroyAllWindows()
    return "Successfully opened the camera"

In [13]:
text_box_test()

'Successfully opened the camera'

In [3]:
#function to calculate angles
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle >180.0:
        angle = 360-angle

    return round(angle)

In [46]:
#path=os.path.join(os.path.dirname(os.getcwd()),"raw_data","new_pushup.mp4")
cap = cv2.VideoCapture('../media/full_pushup.mp4')

# Pushup position variable
stage = 'START' #default is start > DOWN, UP for reps
quality = None #good, bad for angle
rep_counter = 0 # counts number of reps
min_elbow = None # calculate min elbow angle 

#advice variables
advice_shoulder = None # if elbows are too far out
advice_butt = None # if butt is too high
advice_knee = None # if knees are sagging
advice_neck = None # if head is not aligned with back
advice_rep = None # if not full rep
advice_hand = None # if hands are not aligned under shoulders
advice_wrist = None
advice_list = [] # store advice text
advice_counter = 0
advice_start_y = 100

# Setup mediapipe instance
with mp.solutions.pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            # Break the loop if there are no more frames in the video
            break

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # 1. GETTING COORDINATES FOR ALL ANGLES
            ## ALL LEFT BODY PARTS
            l_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            l_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            l_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            l_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            l_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

            ## ALL RIGHT BODY PARTS
            r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            r_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            r_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            r_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            r_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]


            ## 2.1 GETTING ELBOW ANGLES

            # LEFT ELBOW
            # Calculate angle
            left_elbow_angle = calculate_angle(l_shoulder, l_elbow, l_wrist)
            # Visualize angle
            left_elbow_text_position = (image.shape[1] - 200, 90)
            cv2.putText(image, str(f'Left elbow angle: {left_elbow_angle}'),
                           left_elbow_text_position,
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA
                                )

            # RIGHT ELBOW
            # Calculate angle
            right_elbow_angle = calculate_angle(r_shoulder, r_elbow, r_wrist)
            # Visualize angle
            right_elbow_text_position = (image.shape[1] - 200, 120)
            cv2.putText(image, str(f'Right elbow angle: {right_elbow_angle}'),
                           right_elbow_text_position,
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)


            ##2.2 GETTING SHOULDER ANGLES

            #LEFT SHOULDER
            # Calculate angle
            l_shoulder_angle = calculate_angle(l_hip, l_shoulder, l_elbow)
            # Visualize angle
            left_shoulder_text_position = (image.shape[1] - 200, 30)
            cv2.putText(image, str(f'Left shoulder angle: {l_shoulder_angle}'),
                           left_shoulder_text_position,
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA
                                )
            
            # RIGHT SHOULDER
            # Calculate angle
            r_shoulder_angle = calculate_angle(r_hip, r_shoulder, r_elbow)
            # Visualize angle
            right_shoulder_text_position = (image.shape[1] - 200, 60)
            cv2.putText(image, str(f'Right shoulder angle: {r_shoulder_angle}'),
                           right_shoulder_text_position,
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)

            avg_shoulder_angle = round((l_shoulder_angle+r_shoulder_angle)/2)


            ##2.3 GETTING HIPS ANGLES

            # LEFT HIP
            # Calculate angle
            left_hip_angle = calculate_angle(l_shoulder, l_hip, l_ankle)
            # Visualize angle
            left_hip_text_position = (image.shape[1] - 200, 150)
            cv2.putText(image, str(f'Left hip angle: {left_hip_angle}'),
                           left_hip_text_position,
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)

            # RIGHT HIP
            # Calculate angle
            right_hip_angle = calculate_angle(r_shoulder, r_hip, r_ankle)
            # Visualize angle
            right_hip_text_position = (image.shape[1] - 200, 180)
            cv2.putText(image, str(f'Right hip angle: {right_hip_angle}'),
                           right_hip_text_position,
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)


            #3.1 Shoulder quality logic
            if l_shoulder_angle > 50:
                quality = 'BAD'
                advice_shoulder = 'Tuck elbows'
                if not (advice_shoulder in advice_list):
                    advice_list.append(advice_shoulder)
            elif l_shoulder_angle < 50:
                quality = 'GOOD'

            # TODO: Butt quality logic
            advice_butt = 'Lower hips'
            
            # TODO: Knee quality logic
            advice_knee = 'Straighten legs'
            
            # TODO: Neck quality logic
            advice_neck = 'Tuck chin in'
                
            #3.2 Rep Counter and State logic
            if left_elbow_angle and right_elbow_angle < 90:
                stage = "DOWN"
                # print(min_elbow)
            if left_elbow_angle and right_elbow_angle > 110 and stage =='DOWN':
                stage = "UP"
                rep_counter +=1
                
            # TODO: Min Elbow Angle for Rep Advice
            # if stage == "DOWN":
            advice_rep = 'Go deeper'
                
            # text color settings
            if quality == 'BAD':
                q_color = (14, 14, 232)
            if quality == 'GOOD':
                q_color = (31, 194, 53)

            # Visualize right shoulder quality status
            cv2.putText(image, quality, 
                           tuple(np.multiply(r_shoulder, [1280, 720]).astype(int)), 
                           cv2.FONT_HERSHEY_DUPLEX, 1, q_color, 3, cv2.LINE_AA
            ) 
            
            # Visualize right shoulder quality status
            cv2.putText(image, quality, 
                           tuple(np.multiply(l_shoulder, [1280, 720]).astype(int)), 
                           cv2.FONT_HERSHEY_DUPLEX, 1, q_color, 3, cv2.LINE_AA
            )
            
            #visualize vector from shoulders to hands
            cv2.arrowedLine(image, tuple(np.multiply(l_shoulder, [1280, 720]).astype(int)),
                            tuple(np.multiply(l_wrist, [1280, 720]).astype(int)), 
                            (235, 71, 12), 3, cv2.LINE_AA)
            cv2.arrowedLine(image, tuple(np.multiply(r_shoulder, [1280, 720]).astype(int)),
                            tuple(np.multiply(r_wrist, [1280, 720]).astype(int)), 
                            (235, 71, 12), 3, cv2.LINE_AA)
            
            #calculate distance between shoulders
            l_shoulder = np.array([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
                          landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z])

            r_shoulder = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,
                          landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z])
            
            shoulder_distance = np.linalg.norm(l_shoulder - r_shoulder)
            
            #calculate distance between hands
            l_wrist = np.array([landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y,
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].z])
            
            r_wrist = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y,
                          landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].z])
            
            wrist_distance = np.linalg.norm(l_wrist - r_wrist)
            
            # Visualize shoulder/wrist distances
            shoulder_dist_text_position = (image.shape[1] - 200, 200)
            cv2.putText(image, str(f'Shoulder dist: {shoulder_distance}'),
                           shoulder_dist_text_position,
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
            
            wrist_dist_text_position = (image.shape[1] - 200, 215)
            cv2.putText(image, str(f'Wrist dist: {wrist_distance}'),
                           wrist_dist_text_position,
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)

            # print(shoulder_distance, wrist_distance)
            
            #if hands is wider than shoulders, show a warning
            if abs(wrist_distance - shoulder_distance) > .1: 
                advice_hand = "Hands under shoulders"
                if not (advice_hand in advice_list):
                    advice_list.append(advice_hand)              
                        
        except:
            pass

        #4 STATUS BOX

        #4.1 Setup status box
        cv2.rectangle(image, (0,0), (250,80), (245,117,16), -1)

        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_DUPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(rep_counter), 
                    (10,70), 
                    cv2.FONT_HERSHEY_DUPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        #Stage data
        cv2.putText(image, 'STAGE', (75,12),
                   cv2.FONT_HERSHEY_DUPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage,
                   (60,70),
                   cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 3, cv2.LINE_AA)
        
        #4.1 Setup advice box
        if len(advice_list) > 0:
            cv2.rectangle(image, (0, 80), (400,150), (36, 237, 227), -1)
            # advice text
            cv2.putText(image, "ADVICE:", (15,100), 
                    cv2.FONT_HERSHEY_DUPLEX, .5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, advice_list[0], (15,130), 
                    cv2.FONT_HERSHEY_DUPLEX, 1, (0,0,0), 2, cv2.LINE_AA)
            if len(advice_list)>1:
                cv2.rectangle(image, (0, 150), (400,180), (36, 237, 227), -1)
                cv2.putText(image, advice_list[1], (15,170), 
                        cv2.FONT_HERSHEY_DUPLEX, 1, (0,0,0), 2, cv2.LINE_AA)
                if len(advice_list)>2:
                    cv2.putText(image, advice_list[2], (15,190), 
                            cv2.FONT_HERSHEY_DUPLEX, 1, (0,0,0), 2, cv2.LINE_AA)
        
        # TODO: set up automated advice text box
        # while advice_counter != len(advice_list):
        #     cv2.rectangle(image, (0, 80), (400,150), (36, 237, 227), -1)
        #     # Advice Text
        #     cv2.putText(image, "ADVICE:", (15,advice_start_y), 
        #             cv2.FONT_HERSHEY_DUPLEX, .5, (0,0,0), 1, cv2.LINE_AA)
        #     advice_start_y += 40
        #     cv2.putText(image, advice_list[advice_counter], (15, advice_start_y), 
        #             cv2.FONT_HERSHEY_DUPLEX, 1, (0,0,0), 2, cv2.LINE_AA)
        #     advice_counter += 1
            
       

        #4.3 Render detections
        mp.solutions.drawing_utils.draw_landmarks(image, results.pose_landmarks,
                                                  mp.solutions.pose.POSE_CONNECTIONS,
                                                  mp.solutions.drawing_utils.DrawingSpec(
                                                      color=(245, 117, 66), thickness=2, circle_radius=2),
                                                  mp.solutions.drawing_utils.DrawingSpec(
                                                      color=(245, 66, 230), thickness=2, circle_radius=2)
                                                  )


        #5 DISPLAYING WINDOW
        cv2.imshow('Mediapipe Feed', image)

        #6 EXITING SHORTCUT
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    #CLOSING DISPLAY WINDOW
    cap.release()
    cv2.destroyAllWindows()
